# So you've trained a model....now what?

This notebook will walkthrough saving your trained model so you can use it for prediction at a later data without retraining it again i.e. long after you've shutdown your current python kernel. 

## Reference our Data Science Workflow:
You'll see we're in what's called **ML Engineering** which at many companies is an entirely seperate role from the person who designs & trains the model initially. People in this role tend to come from a software engineering background rather than an analytical background like a data scientist. However, there are always exceptions to the rule, and this diagram is just to give you a general idea of how this workflow happens.
<img src='https://storage.googleapis.com/gweb-cloudblog-publish/images/Intro_To_Data_Science_Fhkolds.max-900x900.jpg' />



## Example Model Deployment Process below on your local machine

We won't be connecting to an external registry here, but this gives you a general idea of the flow:

> 1. Store your trained model in pickle object and save it to your local folder (we would always store our model as some object/file we can use later)
> 1. Read this file into a new machine to create an instance of the model with its trained weights. 
> 1. Run predictions from your new instance
> 1. Use those predictions to take action or store those predictions in a database or file to use later

In production settings, you would use your model instance in 1 of 2 use cases:
> 1. **Batch Processing:** Using an orchestrator (e.g. cron, airflow) you would start a computer on some schedule and then run your python code on that computer (e.g. `python my_prediction_job.py`)<br/>
The script would look something like this:<br/>
```python
 import pandas as pd 
 from sklearn import svm
 from joblib import load
 new_data_to_make_predictions = pd.read_csv('new_data_file.csv') # read my new data
 feature_cols = ['feature_1', 'feature_2']
 X_unseen = new_data_to_make_predictions[feature_cols] #specify the features
 model = load('my_model.joblib') #load my model from storage
 y_preds = model.predict(X_unseen) #run predictions
 new_data_to_make_predictions['predicted_y'] = y_preds
 new_data_to_make_predictions.to_csv('preds.csv')#write predictions to a file for future use```
> 1. **Live Inference:** an always on app that you can communicate with through an API or event streaming for real-time predictions. You would load the model from this file (or a file similar to it) into an always on computer (e.g. an ec2 instance, group of instances, containers in kubernetes, etc.). It would be ready to recieve a request that contained a set of features and would return a response with your prediction 


## Batch Processing Example

In [68]:
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y= datasets.load_iris(return_X_y=True)
clf.fit(X, y)

from joblib import dump, load
dump(clf, 'my_model.joblib') 

['my_model.joblib']

In [69]:
some_new_X, _ = datasets.load_iris(return_X_y=True)

clf2 = load('my_model.joblib') 
y_preds = clf2.predict(some_new_X[0:1])
y_preds[0]

0

In [70]:
y_preds = clf.predict(some_new_X[0:1])
y_preds[0]

0

In [71]:
# the feautres we 
some_new_X[0:1]

array([[5.1, 3.5, 1.4, 0.2]])

In [72]:
y_preds.tofile('my_preds_file.csv', sep=',')

# Live Inference App Example

Example App that could be deployed on a remote server:

```python
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify
from joblib import load
from sklearn import svm

app = Flask(__name__)
clf3 = load('my_model.joblib')

@app.route('/api',methods=['POST'])
def predict():
    data = request.get_json(force=True)
    df = pd.io.json.json_normalize(data)
    feature_cols = ['feature_1', 'feature_2', 'feature_3', 'feature_4']
    df['feature_1'] = df['feature_1'].astype(float)
    df['feature_2'] = df['feature_2'].astype(float)
    df['feature_3'] = df['feature_3'].astype(float)
    df['feature_4'] = df['feature_4'].astype(float)
    prediction = clf3.predict(df[feature_cols])
    output = str(prediction[0])
    return jsonify(output)
if __name__ == '__main__':
    app.run(port=5009, debug=True)```
    
This would be in a file called something like `server.py` and be started from the terminal with a command like: ``` python server.py```

In [73]:
import requests
url = 'http://localhost:5010/api'
new_observation = {'feature_1':'5.1','feature_2':'3.5','feature_3':'1.4','feature_4':'0.2'}
print(url)
display(new_observation)

http://localhost:5010/api


{'feature_1': '5.1',
 'feature_2': '3.5',
 'feature_3': '1.4',
 'feature_4': '0.2'}

In [74]:
r = requests.post(url,json=new_observation)
r

<Response [200]>

In [75]:
display(r.json())

'0'